In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
oldDataFrame= pd.read_csv('tts.csv')
ON = oldDataFrame["ON"]
threeMonth = oldDataFrame["3M"]
sixMonth = oldDataFrame["6M"]
oneYear = oldDataFrame["1Y"]
twoYear = oldDataFrame["2Y"]
threeYear = oldDataFrame["3Y"]
fourYear = oldDataFrame["4Y"]
fiveYear = oldDataFrame["5Y"]

In [2]:
def deriveYieldCurve(dateIndex, monthIndex, change): #monthIndex: 0 - 60
    if dateIndex > 228:
        ONPoint = ON[dateIndex]
        threeMonthPoint = threeMonth[dateIndex]
        sixMonthPoint = sixMonth[dateIndex]
        oneYearPoint = oneYear[dateIndex]
        twoYearPoint = twoYear[dateIndex]
        threeYearPoint = threeYear[dateIndex]
        fourYearPoint = fourYear[dateIndex]
        fiveYearPoint = fiveYear[dateIndex]


        if monthIndex == 0:
            return ONPoint + change
        elif monthIndex > 0 and monthIndex <= 3:
            return (monthIndex - 0) * (threeMonthPoint - ONPoint) / 3.0 + ONPoint + change
        elif monthIndex > 3 and monthIndex <= 6:
            return (monthIndex - 3) * (sixMonthPoint - threeMonthPoint) / 3.0 + threeMonthPoint + change
        elif monthIndex > 6 and monthIndex <= 12:
            return (monthIndex - 6) * (oneYearPoint - sixMonthPoint) / 6.0 + sixMonthPoint + change
        elif monthIndex > 12 and monthIndex <= 24:
            return (monthIndex - 12) * (twoYearPoint - oneYearPoint) / 12.0 + oneYearPoint + change
        elif monthIndex > 24 and monthIndex <= 36:
            return (monthIndex - 24) * (threeYearPoint - twoYearPoint) / 12.0 + twoYearPoint + change
        elif monthIndex > 36 and monthIndex <= 48:
            return (monthIndex - 36) * (fourYearPoint - threeYearPoint) / 12.0 + threeYearPoint + change
        elif monthIndex > 48 and monthIndex <= 60:
            return (monthIndex - 48) * (fiveYearPoint - fourYearPoint) / 12.0 + fourYearPoint + change
        
    else:
        ONPoint = ON[dateIndex]
        threeMonthPoint = threeMonth[dateIndex]
        sixMonthPoint = sixMonth[dateIndex]
        oneYearPoint = oneYear[dateIndex]
        twoYearPoint = twoYear[dateIndex]
        threeYearPoint = threeYear[dateIndex]
        fourYearPoint = fourYear[dateIndex]
        fiveYearPoint = fiveYear[dateIndex]


        if monthIndex == 0:
            return ONPoint
        elif monthIndex > 0 and monthIndex <= 3:
            return (monthIndex - 0) * (threeMonthPoint - ONPoint) / 3.0 + ONPoint 
        elif monthIndex > 3 and monthIndex <= 6:
            return (monthIndex - 3) * (sixMonthPoint - threeMonthPoint) / 3.0 + threeMonthPoint
        elif monthIndex > 6 and monthIndex <= 12:
            return (monthIndex - 6) * (oneYearPoint - sixMonthPoint) / 6.0 + sixMonthPoint
        elif monthIndex > 12 and monthIndex <= 24:
            return (monthIndex - 12) * (twoYearPoint - oneYearPoint) / 12.0 + oneYearPoint
        elif monthIndex > 24 and monthIndex <= 36:
            return (monthIndex - 24) * (threeYearPoint - twoYearPoint) / 12.0 + twoYearPoint
        elif monthIndex > 36 and monthIndex <= 48:
            return (monthIndex - 36) * (fourYearPoint - threeYearPoint) / 12.0 + threeYearPoint
        elif monthIndex > 48 and monthIndex <= 60:
            return (monthIndex - 48) * (fiveYearPoint - fourYearPoint) / 12.0 + fourYearPoint
        


In [3]:
def getWeights():
    #weights = {}
    weights = [[], [], []]
    #5% increments
    #for n in range(4620):
    result1 = 0.0
    result2 = 0.0
    result3 = 0.0
    for i in range(21): #i: 0-20
        firstNumber = i * 0.05
        result1 = firstNumber
        residuals = 1.0 - firstNumber
        for j in range(round(residuals / 0.05) + 1): #j: 0-20
            secondNumber = j * 0.05
            #r2 = 1 - r1
            result2 = secondNumber
            #for k in range(int(r2 / 0.05) + 1):
            result3 = 1 - result1 - result2
            weights[0].append(result1)
            weights[1].append(result2)
            weights[2].append(result3)
    #weights[0,n] = result1
    #weights[1,n] = result2
    #weights[2,n] = result3
    return weights

In [4]:
def getMovingAverage(t, i, change):
    averageRate = []
    for j in range(i):
        averageRate.append(deriveYieldCurve(t + 168 - j, i, change))
    return np.mean(averageRate)

In [5]:
def getRowSum(t, i, start, change): #start is the beginning of the tenor we choose
    sumSet = []
    for j in range(start, i + 1):
        sumSet.append(deriveYieldCurve(t + 168, j, change))
    return np.sum(sumSet)

In [6]:
def getElement(i,change,x,y): # t == 0: 201401 , (x,y) starts from (0,0)
    #x = int(x)
    #y = int(y)
    if i == 0:
        return deriveYieldCurve(x + 168, i, change)
    else:
        if x >= y: #requirement
            if y == 0:
                return getMovingAverage(x, i, change)
            else:
                difference = x - y
                if difference >= i:
                    return getMovingAverage(x, i,change)
                else:
                    vertical = 0.0 #except the top one which is difference
                    for j in range(difference):
                        vertical = vertical + deriveYieldCurve(168 + y + 1 + j, i, change)
                    horizon = getRowSum(y, i, difference + 1, 0)
                    return (vertical + horizon) / i

In [7]:
def andysProcessFirstPart(startPoint, totalTime, change):
    dataFrame = oldDataFrame[startPoint:].reset_index() # 168 is 2014-01
    #transferPricingRate = dataFrame["TP_Rate"]
    customerRate = dataFrame["Customer_rate"]
    productBalance = dataFrame["Balance"]
    timeIndex = [0, 3, 6, 12, 24, 36, 48, 60]
    TP_Rate = {}
    firstRow = []
    averageRate = []
    averageRate.append(deriveYieldCurve(168, 0, change))
    firstRow.append(np.mean(averageRate))
    for i in range(1, timeIndex[-1] + 1):
        averageRate = []
        for j in range(i):
            averageRate.append(deriveYieldCurve(168 - j, i, change))
        firstRow.append(np.mean(averageRate))
    TP_Rate[0] = firstRow #line 172
    
    
    for t in range(1, totalTime): # t = 1 means line 173
        #print(t)
        TP_Rate[t] = []
        for i in range(timeIndex[-1] + 1): #0-60
            if i == 0:
                TP_Rate[t].append(getElement(i, change, t, 0))
            else:
                balanceChange = [productBalance[0]]
                for j in range(t):
                    balanceChange.append(productBalance[j+1] - productBalance[j])
                sumUP = 0
                for k in range(t + 1):
                    sumUP = sumUP + balanceChange[k] * getElement(i, change, t, k)
                TP_Rate[t].append(sumUP / productBalance[t])
                
    
    monthlyTransferPricingVectorReturn = {}
    weights = getWeights()
    weightsCombination = {}
    indicators = {}
    timeIndex = [0, 3, 6, 12, 24, 36, 48, 60]
    for t in range(totalTime):
        monthlyTransferPricingVectorReturn[t] = [] # final TP_Rate
        weightsCombination[t] = []
        indicators[t] = []
        for i in range(8):
            for j in range(i + 1, 8):
                for k in range(j + 1, 8):
                    for l in range(len(weights[0])):                   
                        monthlyTransferPricingVectorReturn[t].append(TP_Rate[t][timeIndex[i]] * weights[0][l] + TP_Rate[t][timeIndex[j]] * weights[1][l] + TP_Rate[t][timeIndex[k]] * weights[2][l])                  
                        weightsCombination[t].append([weights[0][l], weights[1][l], weights[2][l]])
                        indicators[t].append([[i,j,k]])
            
    NIM_Liability = {}
    NIM_Assets = {}
    for t in range(totalTime):
        NIM_Liability[t] = []
        NIM_Assets[t] = []
        for i in range(len(monthlyTransferPricingVectorReturn[t])):
            NIM_Liability[t].append(monthlyTransferPricingVectorReturn[t][i] - customerRate[t])
            NIM_Assets[t].append(customerRate[t] - monthlyTransferPricingVectorReturn[t][i])
    
    NIM = {}
    for t in range(totalTime):
        NIM[t] = []
        for i in range(len(monthlyTransferPricingVectorReturn[t])):
            NIM[t].append(NIM_Liability[t][i])
    
    
    newNIM = {} #transpose
    for i in range(len(monthlyTransferPricingVectorReturn[t])):
        newNIM[i] = []
        for t in range(totalTime):
            newNIM[i].append(NIM[t][i])
    
    stdNIM = []
    for i in range(len(monthlyTransferPricingVectorReturn[t])):
        stdNIM.append(np.std(newNIM[i]))
        
    averageNIM = []
    for i in range(len(monthlyTransferPricingVectorReturn[t])):
        averageNIM.append(np.mean(newNIM[i]))
    
    return [stdNIM, indicators, weightsCombination, averageNIM, TP_Rate]
    
    #return TP_Rate, balanceChange

In [8]:
def GOTResult(startPoint, totalTime, change):
    ap = andysProcessFirstPart(startPoint, totalTime, change)
    
    allStd = ap[0]
    index = 0
    minstd = 100000000
    for i in range(len(allStd)):
        if allStd[i] < minstd:
            minstd = allStd[i]
            index = i
    productsName = ["overNight", "threeMonth", "sixMonth", "oneYear", "twoYear", "threeYear", "fourYear", "fiveYear"]        
    portfolioChosen = []
    for j in ap[1][0][index][0]:
        portfolioChosen.append(productsName[j])
    print("The portfolio chosen is:", portfolioChosen, "With weights:", ap[2][0][index], "the standard deviation is", minstd)

    allAverage = ap[3]
    index2 = 0
    maxAverage = 0
    for i in range(len(allAverage)):
        if allAverage[i] >= maxAverage:
            maxAverage = allAverage[i]
            index2 = i
    #productsName = ["overNight", "threeMonth", "sixMonth", "oneYear", "twoYear", "threeYear", "fourYear", "fiveYear"]        
    portfolioChosen2 = []
    for j in ap[1][0][index2][0]:
        portfolioChosen2.append(productsName[j])
    print("The portfolio chosen is:", portfolioChosen2, "With weights:", ap[2][0][index2], "the average is", maxAverage)



In [ ]:
result = GOTResult(168,96,0)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
